In [ ]:
import requests
import lxml.html as lh
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [8]:

url = 'http://www.allocine.fr/film/fichefilm-143692/critiques/spectateurs/'
uri_pages = '?page='
nbPages = 400
tags = ['//span[@class="stareval-note"]', \
        '//div[@class="content-txt review-card-content"]' ]
cols = ['Note', 'Description' ]
 
page = requests.get(url)
doc = lh.fromstring(page.content)
 
def getPage(url):
    page = requests.get(url)
    doc = lh.fromstring(page.content)
 
    # Get the Web data via XPath
    content = []
    for i in range(len(tags)):
        content.append(doc.xpath(tags[i]))
 
    # Gather the data into a Pandas DataFrame array
    df_liste = []
    for j in range(len(tags)):
        tmp = pd.DataFrame([content[j][i].text_content().strip() for i in range(len(content[i]))], columns=[cols[j]])
        tmp['key'] = tmp.index
        df_liste.append(tmp)
 
    # Build the unique Dataframe with one tag (xpath) content per column
    liste = df_liste[0]
    for j in range(len(tags)-1):
        liste = liste.join(df_liste[j+1], on='key', how='left', lsuffix='_l', rsuffix='_r')
        liste['key'] = liste.index
        del liste['key_l']
        del liste['key_r']
     
    return liste
 
# def getPages(_nbPages, _url):
#     liste_finale = pd.DataFrame()
#     for i in range (_nbPages):
#         liste = getPage(_url + uri_pages + str(i+1))
#         liste_finale = pd.concat([liste_finale, liste], ignore_index=True)
#     return liste_finale
 
# liste_totale = getPages(nbPages, url)
# liste_totale.to_csv('./allocine_inception_avis.csv', index=False, quoting=csv.QUOTE_NONNUMERIC)


#### utiliser les expressions régulières 

In [9]:
REMPLACE_SANS_ESPACE = re.compile("[;:!\'?,\"()\[\]]")
REMPLACE_AVEC_ESPACE = re.compile("()|(\-)|(\/)|[.]")
PUR_NOMBRE = re.compile("[0-9]")
  
def setClassBin(i):
    if (float(i.replace(',', '.')) > 3):
        return 1
    else:
        return 0
      
def preprocess(txt):
    txt = [PUR_NOMBRE.sub("", (str(line)).lower()) for line in txt] # retire les nombres (comme les années)
    txt = [line.replace('\n', ' ')  for line in txt] # Retire les \n (retours chariots)
    txt = [REMPLACE_SANS_ESPACE.sub("", line.lower()) for line in txt]
    txt = [REMPLACE_AVEC_ESPACE.sub(" ", line) for line in txt]
    return txt


#### préparation

In [10]:
def remplace(i):
    if (float(i.replace(',', '.')) > 3):
        return 1
    else:
        return 0
 
yList = [remplace(x) for x in X.Note]
y = pd.DataFrame(yList)
X = X.drop('Note', axis=1)

NameError: name 'X' is not defined

#### Utilisation stopwords

In [11]:
X['Description'] = pd.DataFrame(preprocess(X['Description']))
french_stopwords = set(stopwords.words('french'))
filtre_stopfr =  lambda text: [token for token in text if token.lower() not in french_stopwords]
X['Description'] = [' '.join(filtre_stopfr(word_tokenize(item))) for item in X['Description']]


NameError: name 'X' is not defined